In [5]:
from program.setup import *
df = pd.read_csv('PD4SDG Demo data exp.csv')
df.columns = [x.strip().replace(" ","_").replace(".","_").lower() for x in df.columns]

sdg_cols = df.columns.str.contains("sdg")
sdgs = df.ix[:,sdg_cols].copy()
sdgs = sdgs.replace({'Goal ':''}, regex=True).fillna(0).astype('int')

target_cols = df.columns.str.contains("target")
targets = df.ix[:,target_cols].copy()
targets.fillna(0,inplace=True)
def f(x,n=0):
    return x.astype('str').str.split(".").str[n].fillna(0)
targets_main = targets.apply(f,n=0).astype('int')
targets_sub = targets.apply(f,n=1).astype('object')

goals = np.union1d(np.unique(sdgs),np.unique(targets_main))
goals = np.delete(goals,0)

def set_diff(A,B):
    return [np.setdiff1d(a,b) for (a,b) in zip(A,B)]

items_df = {'sdg_goals':[np.unique(r) for r in sdgs.values]
           ,'target_goals':[np.unique(r) for r in targets_main.values]}
item_df = pd.DataFrame(items_df)
item_df.index.name ='item'
item_df['sdgs-targets'] = set_diff(item_df['sdg_goals'],item_df['target_goals'])
item_df['targets-sdgs'] = set_diff(item_df['target_goals'],item_df['sdg_goals'])
display(item_df)

goals_df = {'goals':goals
           ,'sdgs':[np.where(sdgs == g)[0] for g in goals]
           ,'targets':[np.where(targets_main == g)[0] for g in goals]}
#goals_df = pd.DataFrame(goals_df,index=goals)
goals_df = pd.DataFrame(goals_df).set_index('goals')
goals_df['sdgs-targets'] = set_diff(goals_df['sdgs'],goals_df['targets'])
goals_df['targets-sdgs'] = set_diff(goals_df['targets'],goals_df['sdgs'])
display(goals_df)

,sdg_goals,target_goals,sdgs-targets,targets-sdgs
item,,,,
0,"[0, 4, 13, 14]","[0, 4, 13, 14]",[],[]
1,"[0, 13, 14]","[0, 13, 14]",[],[]
2,"[0, 4, 14]","[0, 4, 14]",[],[]
3,"[0, 14]","[0, 14]",[],[]
4,"[0, 1, 2, 14]","[0, 14]","[1, 2]",[]
5,"[0, 14]","[0, 14]",[],[]
6,"[0, 14]","[0, 14]",[],[]
7,"[0, 4, 14]","[0, 4, 14]",[],[]
8,"[0, 1, 4, 13, 14]","[0, 1, 4, 13, 14]",[],[]


,sdgs,targets,sdgs-targets,targets-sdgs
goals,,,,
1,"[4, 8]",[8],[4],[]
2,[4],[],[4],[]
4,"[0, 2, 7, 8, 14]","[0, 2, 7, 8, 14]",[],[]
5,[9],[9],[],[]
6,[9],[9],[],[]
8,[12],[],[12],[]
9,[10],"[10, 10]",[],[]
11,[10],"[10, 10, 10]",[],[]
13,"[0, 1, 8, 10, 14, 15, 16]","[0, 1, 8, 10, 10, 10, 10, 14, 15]",[16],[]
